In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as bs
from pathlib import Path
import time

In [36]:
options = Options()
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_argument('--disable-notification')
# options.add_argument('--disable-gpu')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--no-sandbox')

login_url = "https://adl.edu.tw/login.php"
adl = webdriver.Chrome(options=options)
adl.get(login_url)

In [37]:
# 等待縣市下拉選單出現，並選擇縣市
county_select = WebDriverWait(adl, 10).until(
    EC.presence_of_element_located((By.NAME, "school[0]"))
)
county = "臺中市"
county_select = Select(county_select)
county_select.select_by_value(county)

# 等待區域下拉選單出現，並選擇區域
district_select = WebDriverWait(adl, 10).until(
    EC.presence_of_element_located((By.NAME, "school[1]"))
)
district = "403-西區"
district_select = Select(district_select)
district_select.select_by_value(district)

# 等待學校下拉選單出現，並選擇學校
school_select = WebDriverWait(adl, 10).until(
    EC.presence_of_element_located((By.NAME, "school[2]"))
)
school = "190042-中展國小"
school_select = Select(school_select)
school_select.select_by_value(school)

# 填寫帳號和密碼
username_input = adl.find_element(By.NAME, "username")
password_input = adl.find_element(By.NAME, "password")
username = "t042701"
password = "Aa123456"
username_input.send_keys(username)
password_input.send_keys(password)

In [ ]:
# 手動填寫完驗證碼

# 按下登入按鈕
login_button = adl.find_element(By.ID, "doLogin")
login_button.click()

# 等待5秒，確保新頁面載入完成
adl.implicitly_wait(5)

# 按下確認按鈕
bind_button = adl.find_element(By.ID, "okBtn")
bind_button.click()

In [19]:
# 按下課程總攬按鈕
course_button = adl.find_element(By.XPATH, '//button[text()="課程總覽"]')
course_button.click()
adl.implicitly_wait(3)


# 按下三年級然科學連結
subject_link = adl.find_element(By.LINK_TEXT, "自然科學")
subject_link.click()

In [ ]:
for i in range(1, 11):  # 抓取前10個單元的內容
    # 進入單元頁面
    unit_link_xpath = f"(//a[contains(@href, 'ks_viewunit_new')])[{i}]"
    unit_link = adl.find_element(By.XPATH, unit_link_xpath)
    unit_link.click()

    adl.implicitly_wait(5)  # 等待5秒

    # 找到單元名稱，並建立資料夾
    unit_element = adl.find_element(By.CLASS_NAME, "unit")
    unit_name = unit_element.text.strip()
    print(unit_name)
    unit_name = unit_name.replace("/", "-")
    unit_name = unit_name.replace(":", "-")
    unit_folder = Path(unit_name)
    unit_folder.mkdir(exist_ok=True)

    try:
        subunit_elements = adl.find_elements(
            By.XPATH, "//div[contains(@class, 'snodelisttitle')]"
        )

        for idx, subunit_element in enumerate(subunit_elements):
            subunit_name = subunit_element.text.strip()
            print(subunit_name)
            subunit_name = subunit_name.replace("/", "-")
            subunit_name = subunit_name.replace(":", "-")
            subunit_folder = unit_folder.joinpath(subunit_name)
            subunit_folder.mkdir(exist_ok=True)

            # 找到下一個次單元元素，以此來確定練習題的範圍
            if idx + 1 < len(subunit_elements):
                next_subunit_element = subunit_elements[idx + 1]
                exercise_elements = subunit_element.find_elements(
                    By.XPATH,
                    f"following-sibling::div[following-sibling::div[@class='snodelisttitle'][.='{next_subunit_element.text}']]//dl/dt",
                )
            else:
                exercise_elements = subunit_element.find_elements(
                    By.XPATH,
                    ".//following-sibling::div[@class='snodelist class-list4']//dl/dt",
                )

            for exercise_element in exercise_elements:
                exercise_name = f"{exercise_element.text.strip()} {exercise_element.find_element(By.XPATH, './following-sibling::dd[1]/div[1]').text.strip()}"
                print(exercise_name)
                exercise_name = exercise_name.replace("/", "-")
                exercise_name = exercise_name.replace(":", "-")
                exercise_folder = subunit_folder.joinpath(exercise_name)
                exercise_folder.mkdir(exist_ok=True)

                exercise_button = exercise_element.find_element(
                    By.XPATH, "./following-sibling::dd[1]//a[text()='練習題']"
                )
                exercise_button.click()
                adl.back()

    except NoSuchElementException:
        print("次單元名稱不存在")

    # 返回前一頁，準備點擊下一個連結
    adl.back()